In [1]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Mouse C3KO-RichLi snRNA
####  2023-05-20 by Yi Zhao (Texas Heart Institute, US)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Initiate  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
Ver <- '0'
Step <- 'PART19_Consolidation'
Project <- '2023_neoc3ko_rli'
Code_dir <- paste0('/Volumes/shire/project/', Project, '/code/mouse_v', Ver, '/')

source(paste0(Code_dir, 'src/bioinformatics.R'))
source(paste0(Code_dir, 'src/scRNAseq.R'))
source(paste0(Code_dir, 'src/scATACseq.R'))
source(paste0(Code_dir, 'mouse_v', Ver, '.helper_functions.R'))

InitiateProject('Rivendell', Ver, Step, 'mouse', Project, 'shire')

####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [2]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Load data  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
srt <- readRDS('integrated/PART10.annotated.srt.rds')
srt.backup <- srt
#srt <- srt.backup
#srt$Cell_state <- as.vector(srt$Cell_type)

In [3]:
srt <- srt.backup

In [4]:
srt$Cell_state <- NA

In [5]:
Table(srt$Cell_type)


Cardiomyocyte   Endocardium    Epicardium    Fibroblast       Myeloid 
        56287         13462          6665          9715          1668 
       T cell        B cell         Glial      Pericyte     Adipocyte 
          197           180           142          1723           405 
          SMC           BEC           LEC Contamination       Doublet 
         1068          3007           349           402          6365 

In [6]:
srt$Cell_state[srt$Cell_type %in% c('Endocardium')] <- 'Endocardium'
#srt$Cell_state[srt$Cell_type %in% c('Epicardium')] <- 'Epicardium'
srt$Cell_state[srt$Cell_type %in% c('Pericyte')] <- 'Pericyte'
srt$Cell_state[srt$Cell_type %in% c('Doublet')] <- 'Doublet'
srt$Cell_state[srt$Cell_type %in% c('Adipocyte')] <- 'Adipocyte'
srt$Cell_state[srt$Cell_type %in% c('SMC')] <- 'SMC'
srt$Cell_state[srt$Cell_type %in% c('BEC')] <- 'BEC'
srt$Cell_state[srt$Cell_type %in% c('LEC')] <- 'LEC'
srt$Cell_state[srt$Cell_type %in% c('T cell')] <- 'T cell'
srt$Cell_state[srt$Cell_type %in% c('B cell')] <- 'B cell'
srt$Cell_state[srt$Cell_type %in% c('Glial')] <- 'Glial'
srt$Cell_state[srt$Cell_type %in% c('Contamination')] <- 'Contamination'

In [7]:
emb <- matrix(NA, ncol = 2, nrow = ncol(srt), dimnames = list(Cells(srt), c('subSCVIUMAP_1', 'subSCVIUMAP_2')))
srt@reductions$sub_scVI_umap <- CreateDimReducObject(embeddings = emb, assay = 'RNA', key = 'subSCVIUMAP_')
emb1 <- matrix(NA, ncol = 2, nrow = ncol(srt), dimnames = list(Cells(srt), c('subcleanSCVIUMAP_1', 'subcleanSCVIUMAP_2')))
srt@reductions$sub_clean_scVI_umap <- CreateDimReducObject(embeddings = emb1, assay = 'RNA', key = 'subcleanSCVIUMAP_')


In [8]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Consolidate CMs  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
rds.list <- c('PART11.cardio_cells.srt.rds',
            'PART12.fibro_cells.srt.rds',
            'PART14.epicar_cells.srt.rds',
            'PART15.myeloid_cells.srt.rds')

for(i in 1:4){
#        message(i)
        print(paste0('integrated/',rds.list[[i]]))
        
        cm.srt <- readRDS(paste0('integrated/',rds.list[[i]]))
        
        srt$Cell_state[Cells(cm.srt)] <- as.vector(cm.srt$Cell_state)

        Table(srt$Cell_state, srt$Cell_type)

        srt@reductions$sub_scVI_umap@cell.embeddings[Cells(cm.srt),] <- cm.srt@reductions$sub_scVI_umap@cell.embeddings
        srt@reductions$sub_clean_scVI_umap@cell.embeddings[Cells(cm.srt),] <- cm.srt@reductions$sub_clean_scVI_umap@cell.embeddings

        p1 <- DimPlot2(srt, reduction = 'sub_scVI_umap', group.by = 'Cell_state', split.by = 'Cell_type',
                 ncol = 3, label = T, pt.size = 1, cols = mycol_40)
        p2 <- DimPlot2(srt, reduction = 'sub_clean_scVI_umap', group.by = 'Cell_state', split.by = 'Cell_type',
                 ncol = 3, label = T, pt.size = 1, cols = mycol_40)
        
        PlotPDF(paste0('/3.',rds.list[[i]],'.heat.maker_all_cluster'), 30, 20)
        print(p1 | p2)
        dev.off()
    }

[1] "integrated/PART11.cardio_cells.srt.rds"


Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 45348 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 29 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 45348 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 29 rows containing missing values (`geom_text()`)."


[1] "integrated/PART12.fibro_cells.srt.rds"


Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 35633 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 34 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 35633 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 34 rows containing missing values (`geom_text()`)."


[1] "integrated/PART14.epicar_cells.srt.rds"


Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 28968 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 38 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 29638 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 39 rows containing missing values (`geom_text()`)."


[1] "integrated/PART15.myeloid_cells.srt.rds"


Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 27300 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 11 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 27970 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 12 rows containing missing values (`geom_text()`)."


In [9]:
Table(srt$Cell_type)


Cardiomyocyte   Endocardium    Epicardium    Fibroblast       Myeloid 
        56287         13462          6665          9715          1668 
       T cell        B cell         Glial      Pericyte     Adipocyte 
          197           180           142          1723           405 
          SMC           BEC           LEC Contamination       Doublet 
         1068          3007           349           402          6365 

In [10]:
srt$Cell_state <- factor(srt$Cell_state, levels = c(
        'CM1', 'CM2', 'CM3', 'CM4', 'CM5', 'CM6', 'CM7',
        'Endocardium',
        'EC1','EC2','EC3','EC4',
        'FB1', 'FB2', 'FB3', 'FB4', 'FB5',
        'Mac1', 'Mac2', 'Mac3', 'Granu',
        'T cell', 'B cell',
        'Glial',
        'Pericyte',
        'Adipocyte',
        'SMC',
        'BEC',
        'LEC',
        'Contamination',
        'Doublet'
))

In [11]:
unique(srt$Cell_type)

[1] Epicardium    Glial         Cardiomyocyte Doublet       SMC          
 [6] Endocardium   Fibroblast    Pericyte      Adipocyte     B cell       
[11] Myeloid       BEC           Contamination LEC           T cell       
15 Levels: Cardiomyocyte Endocardium Epicardium Fibroblast Myeloid ... Doublet

In [12]:
srt$Cell_type[srt$Cell_state == 'Doublet'] <- 'Doublet'

In [13]:
srt$Cell_type <- factor(srt$Cell_type, levels = c(
        'Cardiomyocyte',
        'Endocardium',
        'Epicardium',
        'Fibroblast',
        'Myeloid',
        'T cell',
        'B cell',
        'Glial',
        'Pericyte',
        'Adipocyte',
        'SMC',
        'BEC',
        'LEC',
        'Contamination',
        'Doublet'
))

In [14]:
dev.off()
PlotPDF('1.1.heat.celltype_vs_cellstate', 5, 6)
pheatmap(log(Table(srt$Cell_state, srt$Cell_type)+1), cluster_rows = F, cluster_cols = F)
dev.off()

p1 <- DimPlot2(srt, group.by = 'Cell_type', cols = mycol_40, reduction = 'scVI_umap', label = T)
p2 <- DimPlot2(srt, group.by = 'Cell_state', cols = mycol_40, reduction = 'scVI_umap', label = T)
p1 + p2
PlotPDF('1.2.umap.annotated', 18, 7.5)
p1 + p2
dev.off()

null device 
          1

null device 
          1

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



pdf 
  2

In [15]:
srt$Non_ambiguous <- T
srt$Non_ambiguous[srt$Cell_type %in% c('Doublet')] <- F
srt$Non_ambiguous[srt$Cell_state %in% c('Doublet')] <- F
srt$Non_ambiguous[srt$Cell_type %in% c('Contamination')] <- F

Table(srt$Non_ambiguous, srt$Cell_type)
Table(srt$Non_ambiguous, srt$Cell_state)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

       
        Cardiomyocyte Endocardium Epicardium Fibroblast Myeloid T cell B cell
  FALSE             0           0          0          0       0      0      0
  TRUE          56287       13462       5995       9715    1668    197    180
       
        Glial Pericyte Adipocyte   SMC   BEC   LEC Contamination Doublet
  FALSE     0        0         0     0     0     0           402    7035
  TRUE    142     1723       405  1068  3007   349             0       0

       
          CM1   CM2   CM3   CM4   CM5   CM6   CM7 Endocardium   EC1   EC2   EC3
  FALSE     0     0     0     0     0     0     0           0     0     0     0
  TRUE  20709  9440 11590  6986  5059  1509   994       13462  4175  1022   653
       
          EC4   FB1   FB2   FB3   FB4   FB5  Mac1  Mac2  Mac3 Granu T cell
  FALSE     0     0     0     0     0     0     0     0     0     0      0
  TRUE    145  3202  3200  2050   790   473   809   332   246   281    197
       
        B cell Glial Pericyte Adipocyte   SMC   BEC   LEC Contamination Doublet
  FALSE      0     0        0         0     0     0     0           402    7035
  TRUE     180   142     1723       405  1068  3007   349             0       0

In [16]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Re-embed without ambiguous cells
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
clean.srt <- srt[, srt$Non_ambiguous]
clean.srt <- RunUMAP(clean.srt, reduction = 'scVI', dims = 1:50, n.neighbors = 50, min.dist = 0.5,
                     reduction.name = 'clean_umap', reduction.key = 'cUMAP_')

Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"
15:22:07 UMAP embedding parameters a = 0.583 b = 1.334

15:22:07 Read 94198 rows and found 50 numeric columns

15:22:07 Using Annoy for neighbor search, n_neighbors = 50

15:22:07 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

15:22:14 Writing NN index file to temp file /var/folders/5c/r8w5nfxs3j55cvkpkr4kq6q80000gp/T//RtmpVWDxqh/file47b5ebc060d

15:22:14 Searching Annoy index using 1 thread, search_k = 5000

15:22:56 Annoy recall = 100%

15:22:57 Commencing smooth kNN distance calibrati

In [17]:
p3 <- DimPlot2(clean.srt, reduction = 'clean_umap', group.by = 'Cell_type',
         cols = mycol_20, label = T, raster = F)
p4 <- DimPlot2(clean.srt, reduction = 'clean_umap', group.by = 'Cell_state',
                 cols = mycol_30, label = F, raster = F)
PlotPDF('1.2.Re_embed.umap.annotated', 18, 7.5)
p3 | p4
dev.off()

pdf 
  2

In [18]:
## Full umap with doublets and mito-high cells
srt@reductions$full_umap <- srt@reductions$scVI_umap
srt@reductions$full_umap@key <- 'fUMAP_'
colnames(srt@reductions$full_umap@cell.embeddings) <- c('fUMAP_1', 'fUMAP_2')

## Clean umap without doublets or mito-high cells
srt@reductions$clean_umap <- srt@reductions$scVI_umap
srt@reductions$clean_umap@cell.embeddings[,1] <- NA
srt@reductions$clean_umap@cell.embeddings[,2] <- NA
srt@reductions$clean_umap@cell.embeddings[Cells(clean.srt), 1:2] <-
        clean.srt@reductions$clean_umap@cell.embeddings[,1:2]
srt@reductions$clean_umap@key <- 'cUMAP_'
colnames(srt@reductions$clean_umap@cell.embeddings) <- c('cUMAP_1', 'cUMAP_2')

## Sub-umap for all cell types
srt@reductions$sub_umap <- srt@reductions$sub_scVI_umap
srt@reductions$sub_umap@key <- 'subUMAP_'
colnames(srt@reductions$sub_umap@cell.embeddings) <- c('subUMAP_1', 'subUMAP_2')

## Sub-umap for all non-ambiguous cell types 
srt@reductions$sub_clean_umap <- srt@reductions$sub_clean_scVI_umap
srt@reductions$sub_clean_umap@key <- 'subcUMAP_'
colnames(srt@reductions$sub_clean_umap@cell.embeddings) <- c('subcUMAP_1', 'subcUMAP_2')

In [19]:
p5 <- (DimPlot2(srt, reduction = 'clean_umap', group.by = 'Cell_type',
         cols = mycol_20, label = T, raster = T) +
        DimPlot2(srt, reduction = 'full_umap', group.by = 'Cell_type',
                 cols = mycol_20, label = T, raster = T)) 


Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



In [20]:
PlotPDF('1.3.check_embed.umap.annotated', 18, 7.5)
p5
dev.off()


Warning message:
"Removed 7437 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_text()`)."


pdf 
  2

In [22]:
p5 <- (DimPlot2(srt, reduction = 'clean_umap', group.by = 'Cell_type',
         cols = mycol_20, label = T, raster = T) +
        DimPlot2(srt, reduction = 'full_umap', group.by = 'Cell_type',
                 cols = mycol_20, label = T, raster = T)) 

p6 <- DimPlot2(srt, reduction = 'sub_umap', group.by = 'Cell_state', split.by = 'Cell_type',
                 cols = mycol_40, label = T, raster = T, ncol = 6)
PlotPDF('1.3.check_embed.umap.annotated', 18, 7.5)
p5
dev.off()

PlotPDF('1.4.check_embed.umap.annotated', 25, 15)
p6
dev.off()



Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 7437 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_text()`)."


pdf 
  2

Warning message:
"Removed 27300 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 10 rows containing missing values (`geom_text()`)."


pdf 
  2

In [25]:
all_reductions <- srt@reductions
srt2 <- DietSeurat(srt, counts = T, data = T, scale.data = F,
                   assays = c('RNA', 'ATAC'),
                   dimreducs = c('sub_clean_umap', 'sub_umap', 'full_umap', 'clean_umap'))

p1 <- DimPlot2(srt2, reduction = 'full_umap', group.by = 'Cell_type', cols = mycol_20, label = T, repel = TRUE)
p2 <- DimPlot2(srt2, reduction = 'full_umap', group.by = 'Cell_state',
               cols = mycol_40, label = T, repel = TRUE)
p3 <- DimPlot2(srt2, reduction = 'clean_umap', group.by = 'Cell_type', cols = mycol_20, label = T, repel = TRUE)
p4 <- DimPlot2(srt2, reduction = 'clean_umap', group.by = 'Cell_state',
               cols = mycol_40, label = T, repel = TRUE)
p <- wrap_plots(p1, p2, p3, p4, ncol = 2)
PlotPDF('02.1.umap.annotation_clean_umap', 18*0.8, 15*0.8)
p
dev.off()

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 7437 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_text_repel()`)."
Warning message:
"Removed 7437 rows containing missing values (`geom_scattermore()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_text_repel()`)."


pdf 
  2

In [26]:
i <- c(
        'Cardiomyocyte',
        'Endocardium',
        'Epicardium',
        'Fibroblast',
        'Myeloid',
        'T cell',
        'B cell',
        'Glial',
        'Pericyte',
        'Adipocyte',
        'SMC',
        'BEC',
        'LEC',
        'Contamination',
        'Doublet'
      )

plist <- list()
for (j in 1:L(i)){
        message(i[j])
        plist[[j]] <-  DimPlot2(srt2[, srt2$Cell_type == i[j]],
                                reduction = 'sub_clean_umap', group.by = 'Cell_state', cols = mycol_10, label = T)
}
p <- wrap_plots(plist, ncol = 3)
PlotPDF('02.2.umap.annotation_sub_umap', 15, 15)
p
dev.off()
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
saveRDS(all_reductions, 'integrated/PART19.all_reductions.srt_dimreducs.rds')
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Cardiomyocyte

Endocardium

Epicardium

Fibroblast

Myeloid

T cell

B cell

Glial

Pericyte

Adipocyte

SMC

BEC

LEC

Contamination

Doublet

Warning message:
"Removed 13462 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 197 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 180 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 142 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 1723 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 405 rows containing missing values (`geom_

pdf 
  2

In [27]:
saveRDS(srt2, 'integrated/PART19.annotated.srt.rds')

In [28]:
Table(srt2$Cell_type)


Cardiomyocyte   Endocardium    Epicardium    Fibroblast       Myeloid 
        56287         13462          5995          9715          1668 
       T cell        B cell         Glial      Pericyte     Adipocyte 
          197           180           142          1723           405 
          SMC           BEC           LEC Contamination       Doublet 
         1068          3007           349           402          7035 

In [29]:
Table(srt2$group2)


wt_p2m wt_p2s c3_p2m c3_p2s 
 28221  22628  25319  25467 

In [30]:
Table(srt2$study)


2023_NeoC3KO_RLi 
          101635 

In [31]:
p2 <- DimPlot2(srt, group.by = 'group2', cols = mycol_10, reduction = 'clean_umap')
PlotPDF('01.4.cumap.annotated_col_by_disease', 10, 10)
p2
dev.off()


Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 7437 rows containing missing values (`geom_scattermore()`)."


pdf 
  2

In [32]:
p3 <- CountCellBarPlot(srt, group.var = 'group2', stack.var = 'Cell_type', percentage = T,
                       stack.color = c(mycol_30, 'grey20', 'grey35', 'grey50', 'grey90'))$plot
PlotPDF('01.5.bar.annotated_composition', 5, 8)
p3
dev.off()

Warning message:
"Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(stack.var)

  # Now:
  data %>% select(all_of(stack.var))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>."
Warning message:
"Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(group.var)

  # Now:
  data %>% select(all_of(group.var))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>."
`summarise()` has grouped output by 'Cell_type'. You can override using the
`.groups` argument.


pdf 
  2